# Society of Minds(User Proxy)

In [1]:
import asyncio
from autogen_agentchat.ui import Console
from autogen_agentchat.agents import AssistantAgent, SocietyOfMindAgent
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

In [2]:
# Load API key
load_dotenv()
api_key = os.getenv('Google_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gemini-1.5-flash-8b', api_key=api_key)

In [3]:
agent_1 = AssistantAgent(name='my_assistant',model_client=model_client)

In [4]:
response= await agent_1.run(task="Hi how are you?")

In [5]:
print(response.messages[-1].content)

I am doing well, thank you for asking.  How can I help you today?



In [6]:
interviewer = AssistantAgent(
    name="assistant1",
    model_client=model_client,
    system_message=(
        "You are an interviewer specialized in Data Science, Generative AI, Agentic AI, and other AI fields. "
        "Begin the interview by directly asking the first question without waiting for any prior user input. "
        "The interview will have exactly three questions not more than that after third question your role is completed you will get out of loop:"
        "1. Ask the candidate to introduce themselves.\n"
        "2. Based on their first answer, ask a follow-up question.\n"
        "3. Ask one final question based on their second answer.\n"
        "Do not request prior answers before asking the first question."
    )
)


In [7]:
interviewee = UserProxyAgent(
    name ='UserProxy',
    description='you are a user proxy agent',
    input_func=input
)

termination_condition = TextMentionTermination(text='APPROVE')

In [8]:

inner_team = RoundRobinGroupChat([interviewer, interviewee], termination_condition=termination_condition)


In [9]:
society_of_mind_agent = SocietyOfMindAgent(
    name="society_of_mind",
    team=inner_team, 
    model_client=model_client
    )

In [10]:
Validator = AssistantAgent(
    name="assistant3",
    model_client=model_client,
    system_message=(
        "You are a validator who ONLY speaks after the interviewer finishes all questions "
        "and says: 'Interview conducted successfully.'\n"
        "When you receive the candidate's full set of answers, do the following:\n"
        "1. Check if the answers are correct and relevant.\n"
        "2. If correct but improvable, suggest specific improvements.\n"
        "3. If no improvements are needed, respond exactly with: 'Perfect, you are selected.'\n"
        "Once you respond, terminate the process — do not continue the conversation."
    )
)


In [11]:
team = RoundRobinGroupChat([society_of_mind_agent, Validator], max_turns=10)

In [ ]:
stream = team.run_stream(task = "Conduct a short interview")
await Console(stream)

---------- TextMessage (user) ----------
Conduct a short interview
